In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jerez,GT,14.1000,-89.7500,81.05,47,0,3.44,clear sky
1,1,Saldanha,ZA,-33.0117,17.9442,59.05,80,19,13.33,few clouds
2,2,Norman Wells,CA,65.2820,-126.8329,-34.56,67,40,2.30,scattered clouds
3,3,Butaritari,KI,3.0707,172.7902,81.97,78,62,9.57,light rain
4,4,Den Helder,NL,52.9599,4.7593,51.78,95,90,15.01,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  66
What is the maximum temperature you would like for your trip?  88


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jerez,GT,14.1000,-89.7500,81.05,47,0,3.44,clear sky
3,3,Butaritari,KI,3.0707,172.7902,81.97,78,62,9.57,light rain
11,11,East London,ZA,-33.0153,27.9116,67.21,79,91,11.23,overcast clouds
12,12,Mataura,NZ,-46.1927,168.8643,66.47,45,94,9.66,overcast clouds
13,13,Tsabong,BW,-26.0217,22.4010,74.16,53,1,10.56,clear sky
15,15,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,83,40,11.50,scattered clouds
16,16,Sao Joao Da Barra,BR,-21.6403,-41.0511,78.89,88,98,20.24,overcast clouds
17,17,Parrita,CR,9.5201,-84.3225,85.77,85,93,2.30,overcast clouds
18,18,Atuona,PF,-9.8000,-139.0333,79.12,70,8,17.87,clear sky
19,19,Mar Del Plata,AR,-38.0023,-57.5575,69.10,88,90,1.01,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                299
City                   299
Country                296
Lat                    299
Lng                    299
Max Temp               299
Humidity               299
Cloudiness             299
Wind Speed             299
Current Description    299
dtype: int64

In [8]:
# Dropping empty rows
preferred_cities_df = preferred_cities_df.dropna()

In [9]:
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jerez,GT,14.1000,-89.7500,81.05,47,0,3.44,clear sky
3,3,Butaritari,KI,3.0707,172.7902,81.97,78,62,9.57,light rain
11,11,East London,ZA,-33.0153,27.9116,67.21,79,91,11.23,overcast clouds
12,12,Mataura,NZ,-46.1927,168.8643,66.47,45,94,9.66,overcast clouds
13,13,Tsabong,BW,-26.0217,22.4010,74.16,53,1,10.56,clear sky
...,...,...,...,...,...,...,...,...,...,...
670,670,Samarai,PG,-10.6167,150.6667,84.67,67,97,8.86,overcast clouds
676,676,Olinda,BR,-8.0089,-34.8553,82.13,74,40,6.91,scattered clouds
679,679,Neiafu,TO,-18.6500,-173.9833,84.88,74,40,8.05,scattered clouds
682,682,Adre,TD,13.4667,22.2000,70.61,30,81,10.16,broken clouds


In [10]:
# Making sure there are no null values
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jerez,GT,81.05,clear sky,14.1000,-89.7500,
3,Butaritari,KI,81.97,light rain,3.0707,172.7902,
11,East London,ZA,67.21,overcast clouds,-33.0153,27.9116,
12,Mataura,NZ,66.47,overcast clouds,-46.1927,168.8643,
13,Tsabong,BW,74.16,clear sky,-26.0217,22.4010,
15,Arraial Do Cabo,BR,78.76,scattered clouds,-22.9661,-42.0278,
16,Sao Joao Da Barra,BR,78.89,overcast clouds,-21.6403,-41.0511,
17,Parrita,CR,85.77,overcast clouds,9.5201,-84.3225,
18,Atuona,PF,79.12,clear sky,-9.8000,-139.0333,
19,Mar Del Plata,AR,69.10,overcast clouds,-38.0023,-57.5575,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not Found/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Jerez.
Closest hotel is Auto hotel Esmeralda.
------------
Retrieving Results for Index 3: Butaritari.
Closest hotel is Isles Sunset Lodge.
------------
Retrieving Results for Index 11: East London.
Closest hotel is Tu Casa.
------------
Retrieving Results for Index 12: Mataura.
Closest hotel is Ellie's Villa.
------------
Retrieving Results for Index 13: Tsabong.
Closest hotel is Springbok Motel.
------------
Retrieving Results for Index 15: Arraial Do Cabo.
Closest hotel is Pousada Porto Praia.
------------
Retrieving Results for Index 16: Sao Joao Da Barra.
Closest hotel is Pousada Porto De Canoas.
------------
Retrieving Results for Index 17: Parrita.
Closest hotel is Beso del Viento.
------------
Retrieving Results for Index 18: Atuona.
Closest hotel is Villa Enata.
------------
Retrieving Results for Index 19: Mar Del Plata.
Closest hotel is Gran Hotel Mar del Plata.
------------
Retrieving Results for Index 21: Bathsheba.
Closest hotel is Atlantis

In [33]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jerez,GT,81.05,clear sky,14.1000,-89.7500,Auto hotel Esmeralda
3,Butaritari,KI,81.97,light rain,3.0707,172.7902,Isles Sunset Lodge
11,East London,ZA,67.21,overcast clouds,-33.0153,27.9116,Tu Casa
12,Mataura,NZ,66.47,overcast clouds,-46.1927,168.8643,Ellie's Villa
13,Tsabong,BW,74.16,clear sky,-26.0217,22.4010,Springbok Motel
15,Arraial Do Cabo,BR,78.76,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
16,Sao Joao Da Barra,BR,78.89,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
17,Parrita,CR,85.77,overcast clouds,9.5201,-84.3225,Beso del Viento
18,Atuona,PF,79.12,clear sky,-9.8000,-139.0333,Villa Enata
19,Mar Del Plata,AR,69.10,overcast clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
